In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
%matplotlib notebook

import os
import glob
import math
import pandas as pd
import numpy as np
import itertools
import random

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Layer, Conv2D, BatchNormalization, Concatenate, UpSampling2D, MaxPool2D, Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow.keras.backend as K

from IPython.display import SVG

import tensorflow_probability as tfp


from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, normalize
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings('ignore')

from ipywidgets import FloatProgress
from IPython.display import display

import HAR_utils
from HAR_utils import get_activity_index, plot_confusion_matrix, get_Train_Test, timeseries_standardize
from attention import Attention2D

fig_width = 12
plt.rcParams["font.size"] = 7

In [6]:
print('GPU name: ', tf.config.experimental.list_physical_devices('GPU'))

GPU name:  []


In [ ]:
from load_utils import num_labels, time_change, setup_dir, find_xml_filenames, find_csv_filenames

In [ ]:
omizu_path = r"G:\JaimeMorales\Codes\omizunagidori\export\Omizunagidori"
umineko_path = r"G:\JaimeMorales\Codes\omizunagidori\export\Umineko"
database_path = r"G:\JaimeMorales\Codes\omizunagidori\database"
database_l_path = r"G:\JaimeMorales\Codes\omizunagidori\database\labels"
database_o_acc_path = r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori"
database_u_acc_path = r"G:\JaimeMorales\Codes\omizunagidori\database\umineko"

## Load Data

In [ ]:
data = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2022\l_2022_acc.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Jaime\\Documents\\JaimeMorales\\Codes\\HAR_acc\\acc2\\daily_data\\acc_03_r.csv'

In [ ]:
data_df.drop(['LW_x','LW_y','LW_z'],axis=1,inplace=True)

# Separate data

In [ ]:
def get_activity_index(act_dict):
    labels = []
    indices = []
    for k,v in act_dict.items():
        indices.append(k)
        labels.append(v)
    return labels, indices

def check_dict(dic):
    new_dic = dic
    rl = []
    vlm = int(np.mean(list(dic.values()))/10)
    print('vlm',vlm)
    for key,value in dic.items():
        if value <= vlm:
            rl.append(key)
    for k in rl:
        new_dic.pop(k)
    return new_dic

def balance_data(X, Y, activities, ratio_to_min=1.2):
    #XをUnder sampling
    #https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.under_sampling.RandomUnderSampler.html#imblearn.under_sampling.RandomUnderSampler
    target_num_dictionary = {}#クラスごとに所望のインスタンス数を設定
    print("before balancing")
    for key, value in activities.items():#行動ごとにインスタンス数をカウント
        instance_num = np.sum(Y == key, axis=0)
        print("%s: %i"% (key,instance_num))
        target_num_dictionary[key] = instance_num
    #print(target_num_dictionary.items())
    target_num_dictionary=check_dict(target_num_dictionary)
    #print(target_num_dictionary.items())
    
    min_instance_num = min(target_num_dictionary.values())#最小のインスタンス数
    
    for key, value in target_num_dictionary.items():
        if value > min_instance_num:
            target_num_dictionary[key] = min(value, int(min_instance_num*ratio_to_min))#最小のインスタンス数でない場合は、最小のratio_to_min倍にするよう設定
    rus = RandomUnderSampler(sampling_strategy=target_num_dictionary)#Under samplerの構築
    X_resampled, Y_resampled = rus.fit_sample(X, Y)#under sampling実行

    print("after balancing")
    for key, value in activities.items():
        print("%s: %i"% (key,np.sum(Y_resampled == key, axis=0)))
    return X_resampled, Y_resampled

def raw_to_nparray(data, winsize, skip, activities, shuffle=False, undersample=False, us_ratio=1.2):
    fp = FloatProgress(min=0, max=len(data)-winsize)
    fp.value = 0
    display(fp)
    fp.description = "converting to np.array"
    
    features = data.columns[1:-2]#データ系列名のリスト。最初（time）と最後（label）以外
    print(features)
    X = []#データ用リスト
    Y = []#ラベル用リスト
    times = []#タイムスタンプ用リスト
    for i in range(0,len(data)-winsize,skip):#skipだけずらしてウインドウを生成
        window = data[features][i:i+winsize]#winsize幅のウインドウ
        
        cidx = i + int(winsize/2)#ウインドウの中心のインデックス
        X.append(window.values)#データをndarrayにしてリストに追加 (ウインドウ幅,次元数)の形状をもつ。
        Y.append(data["l_val"][cidx])#ウインドウの中心のラベルをリストに追加
        times.append(data["time"][cidx])#ウインドウの中心の時刻をリストに追加
        
        fp.value = fp.value + skip
    
    if undersample: #Undersamplingする場合
        timesteps = X[0].shape[0]
        ndim = X[0].shape[1]
        for i in range(len(X)):
            X[i] = X[i].ravel()#次の行の関数のために2次元のndarrayにする必要があるため、各データを一次元に変換
        X_, Y = balance_data(np.array(X), np.array(Y), activities, us_ratio)
        X_ = X_.reshape(X_.shape[0], timesteps, ndim)#(インスタンス数,ウインドウ幅,次元数)の形状にする
        Y = Y.tolist()
        X = []#リストの形式に戻す(以降でシャッフルするため)
        for idx in range(len(Y)):
            X.append(X_[idx,:,:])
        times = []#時刻は意味をなさなくなったため削除
        
    if shuffle:#シャッフルする場合
        p = list(zip(X,Y))#2つのリストのiterator
        random.shuffle(p)#シャッフル
        X, Y = zip(*p)#２つのリストに反映
        X = list(X)#タプル形式をリストに戻す
        Y = list(Y)
        times = []#時刻は意味をなさなくなったため削除
        
    return np.array(X), Y, times

def timeseries_standardize(trainX, testX):

    ss = preprocessing.StandardScaler()#標準化のためのモジュール
    trainXshape = trainX.shape #元のshapeを保持
    trainX = trainX.reshape(-1, trainX.shape[2]) #2次元のndarrayに変換。(-1,次元数)のshape
    ss.fit(trainX)#学習データから、各次元ごとの平均と分散を計算
    trainX = ss.transform(trainX) #標準化
    trainX = trainX.reshape(trainXshape) #元のshapeに戻す

    testXshape = testX.shape#元のshapeを保持
    testX = testX.reshape(-1, testX.shape[2]) #2次元のndarrayに変換。(-1,次元数)のshape
    testX = ss.transform(testX) #標準化
    testX = testX.reshape(testXshape) #元のshapeに戻す
    
    return trainX, testX



def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()
    
from attention1 import AttentionWithContext

def build_model(input_dim,timesteps,class_num):
    hidden_unit = 20
    model = Sequential()
    model.add(LSTM(hidden_unit, activation="sigmoid", input_shape=(timesteps, input_dim), return_sequences=True))
    #model.add(Dropout(0.2))
    model.add(LSTM(hidden_unit, activation="sigmoid", return_sequences=True))
    #model.add(Dropout(0.2))
    model.add(LSTM(hidden_unit, activation="sigmoid"))
    #model.add(Dense(hidden_unit, activation='tanh'))
    model.add(Dense(class_num, activation='softmax'))
    
    #opt = keras.optimizers.Adam(learning_rate=0.05)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

def ApplyLSTMNetwork(trainX, trainY, testX, testY, activities,nb_epoch = 100, batch_size=32):
    target_names,indices = get_activity_index(activities)
    #Yをone-hot encodingに変換
    lb = LabelBinarizer()
    lb.fit(indices)
    trainY_b = lb.transform(trainY)
    testY_b = lb.transform(testY)
    print(trainY_b)
    #モデルを構築
    model = build_model(trainX.shape[2], trainX.shape[1], len(target_names))
    model.summary()
    #学習・推定
    #エポック数
    #plot_losses = keras_utils.Callbacks() #lossの可視化
    #model.fit(trainX, trainY_b, nb_epoch=nb_epoch, validation_split=0.2, callbacks=[plot_losses], verbose=0)#学習
    model_checkpoint = keras.callbacks.ModelCheckpoint('lstm_membrane.hdf5', monitor='loss',verbose=0, save_best_only=True)
    history = model.fit(trainX, trainY_b, epochs=nb_epoch, validation_split=0.2,callbacks=[model_checkpoint], verbose=1, batch_size=batch_size)
    #history = model.fit(trainX, trainY_b, nb_epoch=nb_epoch, validation_split=0.2, verbose=0)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    #モデル概要
    img = SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))#モデルの可視化
    display(img)
    predict_vec = model.predict(testX)#推定
    predict_class = np.argmax(predict_vec, axis=1)#推定結果をクラスのインデックスの表現（0,1,2,...,len(class_num)）になおす
    predict = [lb.classes_[x] for x in predict_class] #元のラベル（数値）に戻す
    print(predict_class)

    #分類結果
    result = classification_report(testY, predict,labels=indices, target_names=target_names)
    print(result)
    
    #Confusion matrix
    cnf_matrix = confusion_matrix(testY, predict)
    plot_confusion_matrix(cnf_matrix, classes=target_names)

    return predict_vec, lb.classes_

def Train_Test_sep(inn_df,sel=100):
    
    df_len = 0
    
    while df_len == 0:
        if sel == 100:
            selector = random.randint(0,max(list(inn_df['job'])))
        else:
            selector = sel
        
        testdata_df = inn_df[inn_df['job']==selector]
        testdata_df.reset_index(inplace=True)
        testdata_df.drop(['job','index'],axis=1,inplace=True)
    
        traindata_df = inn_df[inn_df['job']!=selector]
        traindata_df.reset_index(inplace=True)
        traindata_df.drop(['job','index'],axis=1,inplace=True)
        df_len = len(testdata_df)
        print('df_len: ', df_len)
    
    
    return testdata_df,traindata_df

In [ ]:
def run_cross_val(cur_acc_dir, save_dir, winsize, skip, lab_d, crossval=10, sels=[1000], nb_epoch = 100, batch_size=32):
    
    cur_acc_df = pd.read_csv(cur_acc_dir)
    act_day = cur_acc_dir[-8:-6]
    newpath = os.path.join(save_dir, act_day)
    
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    
    cur_acc_df.drop(['LW_x','LW_y','LW_z'],axis=1,inplace=True)
    
    if sels[0] == 1000:
        sel_l = [100]*crossval
    else:
        sel_l = sels
    
    for i in range(len(sel_l)):
        print(i)
        sel = sel_l[i]
        
        save_str = 'res_' + act_day + '_' + str(i) + '.csv'
        path_dir = os.path.join(newpath, save_str)
        
        test_data,train_data = Train_Test_sep(cur_acc_df,sel)
    
        trainX, trainY,_ = raw_to_nparray(train_data, winsize, skip, lab_d, True, True,us_ratio=1.5)
        testX, testY, testTimes = raw_to_nparray(test_data, winsize, skip, lab_d)
        trainX, testX = timeseries_standardize(trainX, testX)
    
        predict_vec, classes_ = ApplyLSTMNetwork(trainX, trainY, testX, testY, lab_d, nb_epoch, batch_size)
    
        confidences = np.max(predict_vec, axis=1) 
        predict = [classes_[x] for x in np.argmax(predict_vec, axis=1)]
        test_data_with_conf_pred = pd.merge(test_data, pd.DataFrame(data = {'time':testTimes, 'predicted_label':predict,'confidence':confidences}), on='time')
    
        start_time = test_data_with_conf_pred['time'][0]
        end_time = test_data_with_conf_pred['time'][len(test_data_with_conf_pred)-1]
        Ttimes = list(test_data_with_conf_pred['time'])
        predict1 = list(test_data_with_conf_pred['predicted_label'])
        testYYY = list(test_data_with_conf_pred['l_val'])
    
        labels,indices = get_activity_index(lab_d)
        figs = plt.figure('image',figsize=(16, 6))
        axs = plt.subplot(111)
        axs.plot(Ttimes, predict1, color='r',label='Prediction')
        axs.plot(Ttimes, testYYY, color='g',label='Ground truth')
        axs.set_ylabel('Activity label')
        axs.set_xlabel('time[ms]')
        axs.set_yticks(indices)
        axs.set_yticklabels(labels)
        axs.legend(loc='upper right')
        figs.show()
    
        test_data_with_conf_pred.to_csv(path_dir,index=False)
        print(path_dir)
        print('end'+ str(i))

## Transform Data

In [ ]:
trainX,testX,trainY,testY,testtimes,selector = get_Train_Test(data_df,trn=37)

In [ ]:
print(selector)

In [ ]:
test_data = data_df[data_df['job']==selector]

In [ ]:
test_data

In [ ]:
print('trainX(shape): ',trainX.shape)
print('trainY(shape): ',trainY.shape)

print('testX(shape): ',testX.shape)
print('testY(shape): ',testY.shape)
print('testtimes(len): ',len(testtimes))

In [ ]:
tex = trainX[[5]]
trx = np.delete(trainX,5,0)
print('trainX(shape): ',tex.shape)
print('trainX(shape): ',trx.shape)

In [ ]:
trainX , testX = timeseries_standardize(trainX, testX)

# Network design

In [ ]:
def down_block(x, filters, kernel_size=(1, 3), padding="same", strides=1, name="down_"):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"1")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"2")(c)
    p = MaxPool2D((1, 2),name=name+"M")(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(1, 3), padding="same", strides=1, name="up_"):
    us = UpSampling2D((1, 2),name=name+"U")(x)
    concat = Concatenate()([us, skip])
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"1")(concat)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"2")(c)
    return c

def bottleneck(x, filters, kernel_size=(1, 3), padding="same", strides=1):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name="btl1")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name="btl2")(c)
    c = Dropout(0.5)(c)
    return c

def build_unet_model(height,channels,class_num,f=32):
    
    inputs = Input((1, height, channels))
    
    p0 = inputs
    c1, p1 = down_block(p0, f, name="down_1_")
    c2, p2 = down_block(p1, f*2, name="down_2_")
    c3, p3 = down_block(p2, f*4, name="down_3_")
    p3 = Dropout(0.5)(p3)
    
    bn = bottleneck(p3, f*8)
    
    u1 = up_block(bn, c3, f*4, name="up_1_")
    u2 = up_block(u1, c2, f*2, name="up_2_")
    u3 = up_block(u2, c1, f, name="up_3_")
    
    outputs = Dense(class_num, activation = 'softmax')(u3)
    model = Model(inputs, outputs)
    
    return model

def ApplyUnet(trainX, trainY, testX, testY,TestTimes, activities, nb_epoch=100, fi=32):
    
    target_names,indices = get_activity_index(activities)
    #print(target_names)
    indices.append(11)
    target_names.append('EPadding')
    
    trainY_b = np_utils.to_categorical(trainY, len(target_names))
    testY_b = np_utils.to_categorical(testY, len(target_names))
    lb = LabelBinarizer()
    
    print(indices)
    lb.fit(indices)
    
    model = build_unet_model(trainX.shape[2],trainX.shape[3],len(target_names),f=fi)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
    model.summary()
    
    model_checkpoint = keras.callbacks.ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
    history = model.fit(trainX, trainY_b, epochs=nb_epoch, validation_split=0.2,callbacks=[model_checkpoint], verbose=0)
    
    fig, (tr_plt,cm_plt) = plt.subplots(nrows=2, ncols=1, figsize=(6, 6))
    
    tr_plt.plot(history.history['loss'])
    tr_plt.plot(history.history['val_loss'])
    tr_plt.set_title('Model loss')
    tr_plt.set_ylabel('Loss')
    tr_plt.set_xlabel('Epoch')
    tr_plt.legend(['loss', 'val_loss'], loc='upper left')
      
    predict_v = model.predict(testX)
    predict_vec = predict_v[0][0]
    confidences = np.max(predict_vec, axis=1)
    testYY = testY[0][0]
    
    predict_class = np.argmax(predict_vec, axis=1)
    predict = [lb.classes_[x] for x in predict_class]
    
    print("len(testYY)",len(testYY))
    print("len(predict)",len(predict))
    
    testXX = testX[0][0]
    

    
    for i in range(len(testYY)):
        if testYY[i]==11:
            testYYY = testYY[:i]
            predict1 = predict[:i]
            testXXX = testXX[:i]
            ttimes = TestTimes[:i]
            Confidences = confidences[:i]
            print(i)
            break

    
    '''
    testYYY = testYY
    predict1 = predict
    testXXX = testXX
    ttimes = TestTimes
    Confidences = confidences
    '''
              
    print("len(testYYY)",len(testYYY))
    print("len(predict1)",len(predict1))
    
    result = classification_report(testYYY, predict1, labels=indices, target_names=target_names)
    print(result)
    testXX=testX
    cnf_matrix = confusion_matrix(testYYY, predict1)
    plot_confusion_matrix(cnf_matrix, fig, cm_plt, classes=target_names)
    fig.tight_layout()
    fig.show()
    
    Sloss = history.history['loss']

    return predict1, testYYY, testXXX, Confidences, ttimes, Sloss

## Network testing

In [ ]:
simclass_df = pd.read_csv(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\TF-env\projects\U-net\acc2\labels.csv')
simclass_d = simclass_df['activity'].to_dict()
#simclass_d[11]='pad_class'

In [ ]:
predict1, testYYY, testXXX, Confidences, Ttimes, Sloss = ApplyUnet(trainX, trainY, testX, testY, testtimes, simclass_d, nb_epoch=100, fi=12)

In [ ]:
test_data_with_conf_pred = pd.DataFrame(data = {'time':Ttimes, 'l_val':testYYY, 'predicted_label':predict1,'confidence':Confidences})

In [ ]:
test_data_with_conf_pred.to_csv(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\Test-03-09\U-net\6\res_6_10.csv',index=False)

In [ ]:
tttimes = range(len(predict1))

In [ ]:
labels,indices = get_activity_index(simclass_d)
indices.append(11)
labels.append('EPadding')
figs = plt.figure('result',figsize=(10, 4))
axs = plt.subplot(111)
axs.plot(tttimes, predict1, color='r')
axs.plot(tttimes, testYYY, color='g')
axs.set_ylabel('Activity label')
axs.set_yticks(indices)
axs.set_yticklabels(labels)
figs.show()